# Cook vllm - sglang - test eval script
## OpenAI API for reference
- https://platform.openai.com/docs/api-reference/completions/create
- https://platform.openai.com/docs/api-reference/completions/create

## Cook vllm
- https://docs.vllm.ai/en/latest/getting_started/examples/gradio_webserver.html
- https://docs.vllm.ai/en/latest/getting_started/examples/gradio_openai_chatbot_webserver.html
- https://docs.vllm.ai/en/latest/getting_started/examples/basic.html#gguf

In [ ]:
!vllm serve meta-llama/Llama-3.2-1B-Instruct --dtype auto

## Cook sglang
- https://docs.sglang.ai/backend/server_arguments.html
- > lora_paths: You may provide a list of adapters to your model as a list. Each batch element will get model response with the corresponding lora adapter applied. Currently cuda_graph and radix_attention are not supported with this option so you need to disable them manually. We are still working on through these issues.
- > To enable multi-GPU data parallelism, add --dp 2. Data parallelism is better for throughput if there is enough memory.

In [ ]:
# %pip install "sglang[all]>=0.4.5" --find-links https://flashinfer.ai/whl/cu124/torch2.5/flashinfer-python -q
!python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-1B-Instruct --dp 2 --port 30000

In [ ]:
import requests

url = f"http://localhost:30000/v1/chat/completions"

data = {
    "model": "meta-llama/Llama-3.2-1B-Instruct",
    "messages": [{"role": "user", "content": "Hello em an com chua?"}],
}

response = requests.post(url, json=data)
print(response.json())

## Cook eval script

In [ ]:
# Command to launch SGLang server
!python3 -m sglang.launch_server \
        --model-path "janhq/250404-llama-3.2-3b-instruct-grpo-03-s250" \
        --context-length 8192 \
        --enable-metrics \
        --dtype bfloat16 \
        --host 0.0.0.0 \
        --port 8002 \
        --trust-remote-code \
        --disable-overlap \
        --disable-radix-cache

In [ ]:
# cd scripts/evaluation
!python run_eval.py \
    --config_path eval_config.yaml \
    --method_name research \
    --data_dir {root/path/to/evaluation/data} \
    --dataset_name bamboogle \
    --split test \
    --save_dir {your-save-dir} \
    --save_note research_qwen7b_ins
    --sgl_remote_url {your-launched-sgl-url} \
    --remote_retriever_url {your-hosted-retriever-url} \
    --generator_model {your-local-model-path} \
    --apply_chat True